<a href="https://colab.research.google.com/github/kiranpantha/AIAC-536-NLP-NEPALI-TEXT-SUMMERIZE/blob/main/Summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extractive Text Summary using tfidf representation of text

#Text Analysed
काठमाडौँ — प्रधानमन्त्री पुष्पकमल दाहालले दूरसञ्चार सेवा प्रदायक कम्पनी एनसेलको मुलुकबाहिरै ८० प्रतिशत सेयर खरिद–बिक्रीको विषयमा तत्काल अध्ययन गर्न सम्बन्धित निकायलाई निर्देशन दिएका छन् । प्रधानमन्त्री दाहालको सचिवालयकाअनुसार राजश्व विभाग लगायतका निकायलाई एनसेलको सेयर खरिदको छानबिन गर्न निर्देशन दिएका हुन् ।

एनसेलको सेयर बिक्रीबारे अध्ययन गर्न प्रधानमन्त्रीको निर्देशन
प्रधानमन्त्री दाहालले सोमबार एनसेलको सेयर बिक्रीबारे चासो व्यक्त गर्दै अध्ययनको निर्देशन दिएका हुन् । 'एनसेलको सेयर बिक्रीका सम्बन्धमा प्रकाशित समाचारहरूप्रति प्रधानमन्‍त्री पुष्पकमल दाहालले गम्भीर चासो व्यक्त गर्नुभएको छ,' प्रधानमन्त्रीको सचिवालयले भनेको छ, 'प्रधानमन्‍त्रीले यस सम्बन्धमा प्रचलित कानुनी प्रावधानको पालना भए नभएको, सेयर बिक्री वितरण गर्दा प्रचलित विधि र प्रक्रिया पालना भए नभएको एवं यसबाट नेपाल नेपालको दूरसञ्‍चार क्षेत्र र विदेशी लगानीमा पर्ने असर तथा नेपाल सरकारको राजस्वलगायतका विषयमा पर्ने प्रभावका बारेमा सम्बन्धित निकायलाई शीघ्र अध्ययन गर्न निर्देशन दिनुभएको छ ।'

मलेसियन कम्पनी ‘आजियटा ग्रुप बर्हाड’ले गत शुक्रबार एनसेलको ८० प्रतिशत शेयर स्वामित्व अर्को कम्पनी ‘स्पेक्ट्रलाइट युके लिमिटेड’लाई बेच्ने निर्णय गरेको थियो । नियामक निकायलाई समेत जानकारी नहुने गरी मुलुकबाहिरबाट शेयर खरिद–बिक्री भएको भन्दै यो व्यावसायिक कारोबारको कानुनी प्रक्रियाको वैधता र आर्थिक पादर्शितामाथि प्रश्न उठिरहेका छन् ।

एनसेलको सेयर बिक्रीबारे संघीय संसद्को सार्वजनिक लेखा समितिले पनि चासो व्यक्त गरेको थियो । दूरसञ्चार सेवा प्रदायक कम्पनी र वैदेशिक लगानीका सन्दर्भमा प्रत्यक्ष सरोकार राख्ने नेपालस्थित सरकारी निकायलाई जानकारी नै नदिई सेयर खरिद–बिक्री भएपछि समितिले चासो राख्दै सरोकारवाला निकायलाई लेखा समितिले प्रश्न गर्दै विवरण माग गरिसकेको छ । यस विषयमा प्रधानमन्त्रीले पनि चासो व्यक्त गर्दै तत्काल अध्ययन गर्न निर्देशन दिएका हुन् ।

# Importing necessary libraries and Downloading stopwords.

In [74]:

import math
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Tokenize sentences and create a word frequency matrix for each sentence

In [75]:
def create_word_frequency_matrix(sentences):
    word_frequency_matrix = {}
    stop_words = set(stopwords.words("nepali"))

    for sent in sentences:
        word_freq_table = {}
        words = sent.split()
        for word in words:
            if word in stop_words:
                continue

            if word in word_freq_table:
                word_freq_table[word] += 1
            else:
                word_freq_table[word] = 1

        word_frequency_matrix[sent[:10]] = word_freq_table

    return word_frequency_matrix

# Create a term frequency matrix from the word frequency matrix

In [76]:
def create_term_frequency_matrix(word_frequency_matrix):
    term_frequency_matrix = {}

    for sent, word_freq_table in word_frequency_matrix.items():
        term_freq_table = {}

        count_words_in_sentence = len(word_freq_table)
        for word, count in word_freq_table.items():
            term_freq_table[word] = count / count_words_in_sentence

        term_frequency_matrix[sent] = term_freq_table

    return term_frequency_matrix

# Create a table of how many documents contain each word

In [77]:
def create_document_frequency_table(word_frequency_matrix):
    doc_frequency_table = {}

    for sent, word_freq_table in word_frequency_matrix.items():
        for word, count in word_freq_table.items():
            if word in doc_frequency_table:
                doc_frequency_table[word] += 1
            else:
                doc_frequency_table[word] = 1

    return doc_frequency_table

# Create an inverse document frequency matrix from the word frequency matrix and document frequency table

In [78]:
def create_inverse_document_frequency_matrix(word_frequency_matrix, doc_frequency_table, total_documents):
    inverse_document_frequency_matrix = {}

    for sent, word_freq_table in word_frequency_matrix.items():
        inverse_doc_freq_table = {}

        for word in word_freq_table.keys():
            inverse_doc_freq_table[word] = math.log10(total_documents / float(doc_frequency_table[word]))

        inverse_document_frequency_matrix[sent] = inverse_doc_freq_table

    return inverse_document_frequency_matrix

# Create a term frequency-inverse document frequency matrix from the term frequency matrix and inverse document frequency matrix

In [79]:
def create_term_frequency_inverse_document_frequency_matrix(term_frequency_matrix, inverse_document_frequency_matrix):
    tf_idf_matrix = {}

    for (sent1, term_freq_table1), (sent2, inverse_doc_freq_table2) in zip(term_frequency_matrix.items(), inverse_document_frequency_matrix.items()):
        tf_idf_table = {}

        for (word1, value1), (word2, value2) in zip(term_freq_table1.items(), inverse_doc_freq_table2.items()):
            tf_idf_table[word1] = float(value1 * value2)

        tf_idf_matrix[sent1] = tf_idf_table

    return tf_idf_matrix

# Calculate sentence scores based on TF-IDF

In [80]:
def calculate_sentence_scores(tf_idf_matrix):
    sentence_scores = {}

    for sent, tf_idf_table in tf_idf_matrix.items():
        total_score_per_sentence = 0

        count_words_in_sentence = len(tf_idf_table)
        for word, score in tf_idf_table.items():
            total_score_per_sentence += score
        if count_words_in_sentence != 0:
            sentence_scores[sent] = total_score_per_sentence / count_words_in_sentence
        else:
            sentence_scores[sent] = 0
    return sentence_scores

# Calculate the average score of sentences

In [81]:
def calculate_average_sentence_score(sentence_scores):
    """Calculate the average score of sentences."""
    sum_values = sum(sentence_scores.values())
    average_score = (sum_values / len(sentence_scores))
    return average_score


# Generate a summary based on the sentence scores and a given threshold

In [100]:
def generate_summary(sentences, sentence_scores, threshold):
    """Generate the summary based on the sentence scores."""
    summary = []

    for sentence in sentences:
        if sentence[:10] in sentence_scores and sentence_scores[sentence[:10]] >= (threshold):
            summary.append(sentence)

    return '।'.join(summary)

In [101]:
# User input for the text to be summarized
text = input("Enter Text to be Summarized:\n\n")

Enter Text to be Summarized:

काठमाडौँ — प्रधानमन्त्री पुष्पकमल दाहालले दूरसञ्चार सेवा प्रदायक कम्पनी एनसेलको मुलुकबाहिरै ८० प्रतिशत सेयर खरिद–बिक्रीको विषयमा तत्काल अध्ययन गर्न सम्बन्धित निकायलाई निर्देशन दिएका छन् । प्रधानमन्त्री दाहालको सचिवालयकाअनुसार राजश्व विभाग लगायतका निकायलाई एनसेलको सेयर खरिदको छानबिन गर्न निर्देशन दिएका हुन् ।  एनसेलको सेयर बिक्रीबारे अध्ययन गर्न प्रधानमन्त्रीको निर्देशन  प्रधानमन्त्री दाहालले सोमबार एनसेलको सेयर बिक्रीबारे चासो व्यक्त गर्दै अध्ययनको निर्देशन दिएका हुन् । 'एनसेलको सेयर बिक्रीका सम्बन्धमा प्रकाशित समाचारहरूप्रति प्रधानमन्‍त्री पुष्पकमल दाहालले गम्भीर चासो व्यक्त गर्नुभएको छ,' प्रधानमन्त्रीको सचिवालयले भनेको छ, 'प्रधानमन्‍त्रीले यस सम्बन्धमा प्रचलित कानुनी प्रावधानको पालना भए नभएको, सेयर बिक्री वितरण गर्दा प्रचलित विधि र प्रक्रिया पालना भए नभएको एवं यसबाट नेपाल नेपालको दूरसञ्‍चार क्षेत्र र विदेशी लगानीमा पर्ने असर तथा नेपाल सरकारको राजस्वलगायतका विषयमा पर्ने प्रभावका बारेमा सम्बन्धित निकायलाई शीघ्र अध्ययन गर्न निर्देशन दिनुभएको छ ।'  मलेसियन क

In [102]:
# Tokenizing sentences
sentences = re.split('।', text)
print(sentences)

['काठमाडौँ — प्रधानमन्त्री पुष्पकमल दाहालले दूरसञ्चार सेवा प्रदायक कम्पनी एनसेलको मुलुकबाहिरै ८० प्रतिशत सेयर खरिद–बिक्रीको विषयमा तत्काल अध्ययन गर्न सम्बन्धित निकायलाई निर्देशन दिएका छन् ', ' प्रधानमन्त्री दाहालको सचिवालयकाअनुसार राजश्व विभाग लगायतका निकायलाई एनसेलको सेयर खरिदको छानबिन गर्न निर्देशन दिएका हुन् ', '  एनसेलको सेयर बिक्रीबारे अध्ययन गर्न प्रधानमन्त्रीको निर्देशन  प्रधानमन्त्री दाहालले सोमबार एनसेलको सेयर बिक्रीबारे चासो व्यक्त गर्दै अध्ययनको निर्देशन दिएका हुन् ', " 'एनसेलको सेयर बिक्रीका सम्बन्धमा प्रकाशित समाचारहरूप्रति प्रधानमन्\u200dत्री पुष्पकमल दाहालले गम्भीर चासो व्यक्त गर्नुभएको छ,' प्रधानमन्त्रीको सचिवालयले भनेको छ, 'प्रधानमन्\u200dत्रीले यस सम्बन्धमा प्रचलित कानुनी प्रावधानको पालना भए नभएको, सेयर बिक्री वितरण गर्दा प्रचलित विधि र प्रक्रिया पालना भए नभएको एवं यसबाट नेपाल नेपालको दूरसञ्\u200dचार क्षेत्र र विदेशी लगानीमा पर्ने असर तथा नेपाल सरकारको राजस्वलगायतका विषयमा पर्ने प्रभावका बारेमा सम्बन्धित निकायलाई शीघ्र अध्ययन गर्न निर्देशन दिनुभएको छ ", "'  मलेसियन कम

In [103]:
# Creating word frequency matrix
word_frequency_matrix = create_word_frequency_matrix(sentences)
print(word_frequency_matrix)

{'काठमाडौँ —': {'काठमाडौँ': 1, '—': 1, 'प्रधानमन्त्री': 1, 'पुष्पकमल': 1, 'दाहालले': 1, 'दूरसञ्चार': 1, 'सेवा': 1, 'प्रदायक': 1, 'कम्पनी': 1, 'एनसेलको': 1, 'मुलुकबाहिरै': 1, '८०': 1, 'प्रतिशत': 1, 'सेयर': 1, 'खरिद–बिक्रीको': 1, 'विषयमा': 1, 'अध्ययन': 1, 'सम्बन्धित': 1, 'निकायलाई': 1, 'निर्देशन': 1, 'दिएका': 1}, ' प्रधानमन्': {'प्रधानमन्त्री': 1, 'दाहालको': 1, 'सचिवालयकाअनुसार': 1, 'राजश्व': 1, 'विभाग': 1, 'लगायतका': 1, 'निकायलाई': 1, 'एनसेलको': 1, 'सेयर': 1, 'खरिदको': 1, 'छानबिन': 1, 'निर्देशन': 1, 'दिएका': 1}, '  एनसेलको ': {'एनसेलको': 1, 'सेयर': 1, 'बिक्रीबारे': 1, 'संघीय': 1, 'संसद्को': 1, 'सार्वजनिक': 1, 'लेखा': 1, 'समितिले': 1, 'चासो': 1, 'व्यक्त': 1}, " 'एनसेलको ": {"'एनसेलको": 1, 'सेयर': 2, 'बिक्रीका': 1, 'सम्बन्धमा': 2, 'प्रकाशित': 1, 'समाचारहरूप्रति': 1, 'प्रधानमन्\u200dत्री': 1, 'पुष्पकमल': 1, 'दाहालले': 1, 'गम्भीर': 1, 'चासो': 1, 'व्यक्त': 1, 'गर्नुभएको': 1, "छ,'": 1, 'प्रधानमन्त्रीको': 1, 'सचिवालयले': 1, 'भनेको': 1, 'छ,': 1, "'प्रधानमन्\u200dत्रीले": 1, 'प्रचलित': 2, 'कानुन

In [104]:
# Creating term frequency matrix
term_frequency_matrix = create_term_frequency_matrix(word_frequency_matrix)
print(term_frequency_matrix)

{'काठमाडौँ —': {'काठमाडौँ': 0.047619047619047616, '—': 0.047619047619047616, 'प्रधानमन्त्री': 0.047619047619047616, 'पुष्पकमल': 0.047619047619047616, 'दाहालले': 0.047619047619047616, 'दूरसञ्चार': 0.047619047619047616, 'सेवा': 0.047619047619047616, 'प्रदायक': 0.047619047619047616, 'कम्पनी': 0.047619047619047616, 'एनसेलको': 0.047619047619047616, 'मुलुकबाहिरै': 0.047619047619047616, '८०': 0.047619047619047616, 'प्रतिशत': 0.047619047619047616, 'सेयर': 0.047619047619047616, 'खरिद–बिक्रीको': 0.047619047619047616, 'विषयमा': 0.047619047619047616, 'अध्ययन': 0.047619047619047616, 'सम्बन्धित': 0.047619047619047616, 'निकायलाई': 0.047619047619047616, 'निर्देशन': 0.047619047619047616, 'दिएका': 0.047619047619047616}, ' प्रधानमन्': {'प्रधानमन्त्री': 0.07692307692307693, 'दाहालको': 0.07692307692307693, 'सचिवालयकाअनुसार': 0.07692307692307693, 'राजश्व': 0.07692307692307693, 'विभाग': 0.07692307692307693, 'लगायतका': 0.07692307692307693, 'निकायलाई': 0.07692307692307693, 'एनसेलको': 0.07692307692307693, 'सेयर

In [105]:
# Counting documents per words
doc_frequency_table = create_document_frequency_table(word_frequency_matrix)
print(doc_frequency_table)

{'काठमाडौँ': 1, '—': 1, 'प्रधानमन्त्री': 2, 'पुष्पकमल': 2, 'दाहालले': 2, 'दूरसञ्चार': 2, 'सेवा': 2, 'प्रदायक': 2, 'कम्पनी': 3, 'एनसेलको': 4, 'मुलुकबाहिरै': 1, '८०': 2, 'प्रतिशत': 2, 'सेयर': 5, 'खरिद–बिक्रीको': 1, 'विषयमा': 3, 'अध्ययन': 3, 'सम्बन्धित': 2, 'निकायलाई': 5, 'निर्देशन': 4, 'दिएका': 3, 'दाहालको': 1, 'सचिवालयकाअनुसार': 1, 'राजश्व': 1, 'विभाग': 1, 'लगायतका': 1, 'खरिदको': 1, 'छानबिन': 1, 'बिक्रीबारे': 1, 'संघीय': 1, 'संसद्को': 1, 'सार्वजनिक': 1, 'लेखा': 2, 'समितिले': 2, 'चासो': 4, 'व्यक्त': 3, "'एनसेलको": 1, 'बिक्रीका': 1, 'सम्बन्धमा': 1, 'प्रकाशित': 1, 'समाचारहरूप्रति': 1, 'प्रधानमन्\u200dत्री': 1, 'गम्भीर': 1, 'गर्नुभएको': 1, "छ,'": 1, 'प्रधानमन्त्रीको': 1, 'सचिवालयले': 1, 'भनेको': 1, 'छ,': 1, "'प्रधानमन्\u200dत्रीले": 1, 'प्रचलित': 1, 'कानुनी': 2, 'प्रावधानको': 1, 'पालना': 1, 'नभएको,': 1, 'बिक्री': 1, 'वितरण': 1, 'गर्दा': 1, 'विधि': 1, 'प्रक्रिया': 1, 'नभएको': 1, 'एवं': 1, 'यसबाट': 1, 'नेपाल': 1, 'नेपालको': 1, 'दूरसञ्\u200dचार': 1, 'क्षेत्र': 1, 'विदेशी': 1, 'लगानीमा': 1, 'पर

In [106]:
# Creating inverse document frequency matrix
inverse_document_frequency_matrix = create_inverse_document_frequency_matrix(word_frequency_matrix, doc_frequency_table, len(sentences))
print(inverse_document_frequency_matrix)

{'काठमाडौँ —': {'काठमाडौँ': 1.0, '—': 1.0, 'प्रधानमन्त्री': 0.6989700043360189, 'पुष्पकमल': 0.6989700043360189, 'दाहालले': 0.6989700043360189, 'दूरसञ्चार': 0.6989700043360189, 'सेवा': 0.6989700043360189, 'प्रदायक': 0.6989700043360189, 'कम्पनी': 0.5228787452803376, 'एनसेलको': 0.3979400086720376, 'मुलुकबाहिरै': 1.0, '८०': 0.6989700043360189, 'प्रतिशत': 0.6989700043360189, 'सेयर': 0.3010299956639812, 'खरिद–बिक्रीको': 1.0, 'विषयमा': 0.5228787452803376, 'अध्ययन': 0.5228787452803376, 'सम्बन्धित': 0.6989700043360189, 'निकायलाई': 0.3010299956639812, 'निर्देशन': 0.3979400086720376, 'दिएका': 0.5228787452803376}, ' प्रधानमन्': {'प्रधानमन्त्री': 0.6989700043360189, 'दाहालको': 1.0, 'सचिवालयकाअनुसार': 1.0, 'राजश्व': 1.0, 'विभाग': 1.0, 'लगायतका': 1.0, 'निकायलाई': 0.3010299956639812, 'एनसेलको': 0.3979400086720376, 'सेयर': 0.3010299956639812, 'खरिदको': 1.0, 'छानबिन': 1.0, 'निर्देशन': 0.3979400086720376, 'दिएका': 0.5228787452803376}, '  एनसेलको ': {'एनसेलको': 0.3979400086720376, 'सेयर': 0.30102999566398

In [107]:
# Creating term frequency-inverse document frequency matrix
tf_idf_matrix = create_term_frequency_inverse_document_frequency_matrix(term_frequency_matrix, inverse_document_frequency_matrix)
print(tf_idf_matrix)

{'काठमाडौँ —': {'काठमाडौँ': 0.047619047619047616, '—': 0.047619047619047616, 'प्रधानमन्त्री': 0.0332842859207628, 'पुष्पकमल': 0.0332842859207628, 'दाहालले': 0.0332842859207628, 'दूरसञ्चार': 0.0332842859207628, 'सेवा': 0.0332842859207628, 'प्रदायक': 0.0332842859207628, 'कम्पनी': 0.024898987870492266, 'एनसेलको': 0.01894952422247798, 'मुलुकबाहिरै': 0.047619047619047616, '८०': 0.0332842859207628, 'प्रतिशत': 0.0332842859207628, 'सेयर': 0.014334761698284819, 'खरिद–बिक्रीको': 0.047619047619047616, 'विषयमा': 0.024898987870492266, 'अध्ययन': 0.024898987870492266, 'सम्बन्धित': 0.0332842859207628, 'निकायलाई': 0.014334761698284819, 'निर्देशन': 0.01894952422247798, 'दिएका': 0.024898987870492266}, ' प्रधानमन्': {'प्रधानमन्त्री': 0.05376692341046299, 'दाहालको': 0.07692307692307693, 'सचिवालयकाअनुसार': 0.07692307692307693, 'राजश्व': 0.07692307692307693, 'विभाग': 0.07692307692307693, 'लगायतका': 0.07692307692307693, 'निकायलाई': 0.02315615351261394, 'एनसेलको': 0.030610769897849048, 'सेयर': 0.02315615351261

In [108]:
# Calculating sentence scores
sentence_scores = calculate_sentence_scores(tf_idf_matrix)
print(sentence_scores)

{'काठमाडौँ —': 0.031247585099359543, ' प्रधानमन्': 0.05692182697212068, '  एनसेलको ': 0.07017728766960432, " 'एनसेलको ": 0.020025394503008453, "'  मलेसियन": 0.05105125775382954, ' नियामक नि': 0.04875271610291382, ' दूरसञ्चार': 0.03170732342931205, ' यस विषयमा': 0.07933459180541684, '': 0}


In [109]:
# Finding average sentence score
threshold = calculate_average_sentence_score(sentence_scores)
print(threshold)

0.04324644259284058


In [110]:
# Generating summary
summary = generate_summary(sentences, sentence_scores, 0.8 * threshold)
print(summary)

 प्रधानमन्त्री दाहालको सचिवालयकाअनुसार राजश्व विभाग लगायतका निकायलाई एनसेलको सेयर खरिदको छानबिन गर्न निर्देशन दिएका हुन् ।  एनसेलको सेयर बिक्रीबारे अध्ययन गर्न प्रधानमन्त्रीको निर्देशन  प्रधानमन्त्री दाहालले सोमबार एनसेलको सेयर बिक्रीबारे चासो व्यक्त गर्दै अध्ययनको निर्देशन दिएका हुन् ।'  मलेसियन कम्पनी ‘आजियटा ग्रुप बर्हाड’ले गत शुक्रबार एनसेलको ८० प्रतिशत शेयर स्वामित्व अर्को कम्पनी ‘स्पेक्ट्रलाइट युके लिमिटेड’लाई बेच्ने निर्णय गरेको थियो । नियामक निकायलाई समेत जानकारी नहुने गरी मुलुकबाहिरबाट शेयर खरिद–बिक्री भएको भन्दै यो व्यावसायिक कारोबारको कानुनी प्रक्रियाको वैधता र आर्थिक पादर्शितामाथि प्रश्न उठिरहेका छन् ।  एनसेलको सेयर बिक्रीबारे संघीय संसद्को सार्वजनिक लेखा समितिले पनि चासो व्यक्त गरेको थियो । यस विषयमा प्रधानमन्त्रीले पनि चासो व्यक्त गर्दै तत्काल अध्ययन गर्न निर्देशन दिएका हुन् 


In [111]:
print(f"Length of the summary: {len(summary)}")

Length of the summary: 800
